<a href="https://colab.research.google.com/github/sudhanshuranjansr/WP/blob/master/WP_chats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pandas import Series,DataFrame
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Import spaCy and load the language library
import spacy
from nltk.corpus import stopwords 
nlp = spacy.load('en_core_web_sm')

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,Dropout
from keras.preprocessing.sequence import pad_sequences
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
pip install indic-transliteration

     |████████████████████████████████| 102kB 2.9MB/s 
     |████████████████████████████████| 911kB 10.6MB/s 


In [ ]:
import indic_transliteration
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate

**importing data **

In [ ]:
with open('WhatsApp Chat with Gupta_kapoor prob discuss (1).txt','r') as txt:
    train=txt.readlines()

In [ ]:
with open('Hinglish_to_english2.txt','r') as txt:
    he2=txt.readlines()
with open('hinglish_abusive.txt','r') as txt:
    he_a=txt.readlines()
with open('hinglish_to_english1.txt','r') as txt:
    he1=txt.readlines()
with open('stopwords.txt','r') as txt:
    he_s=txt.readlines()
with open('s_h_w.txt','r') as txt:
    shw=txt.readlines()

In [ ]:
#the way we speak ,the words we use tells a lot about our charatcter and nature and can also prdict our response to the questions at the gievn situation
#here i'm gonna import the chat from wp and trace the charteers upon the besis of the same 
#may have to creat a hinglish dict 

In [ ]:
#function_box

def mes_sep(df,n):
  a=[]
  b=[]
  for  i in df.index:
        if df["Name"][i]==n:
          a=np.append(a,df["Messages"][i])
          b=np.append(b,df["Time"][i])
  dfab=DataFrame([b,a],index=["Time","Message"]).T
  return(dfab)


def detect_language(character):
    maxchar = max(character)
    if u'\u0900' <= maxchar <= u'\u097f':
        return 'hin'
    else:
        return 'eng'


def separator(mat):                                 #separtes hinglish words from the english ones 
  mv2=[]
  mv1=[]
  for i in np.arange(len(mat)-1):
      mv=mat[i].split(",")
      mv11=mv[0]
      mv21=mv[1][:-1]
      mv2=np.append(mv2,mv21)
      mv1=np.append(mv1,mv11)
  dfmv=DataFrame([mv1,mv2],index=["hinglish","english"]).T
  return(dfmv)

def filter(txt):                                    #fiters the dictionary for the given biginninng letter of the Hinglish word
    txt1='^'+txt
    dfl=dfhe['hinglish'].str.contains(txt1)
    dfhe3=dfhe
    dfhe3["logic"]=dfl
    dfr=dfhe3[dfhe3['logic'] == True]
    dfr=dfr.drop("logic",axis=1)
    return(dfr)

def finder(dc,wrd):                                 #finds a prticular word in  a given dictionary
     n=len(dc["english"])
     for i in dc.index.values:
       if (dc["hinglish"][i] == wrd) or (dc["hinglish"][i] in wrd):
         r=dc["english"][i]
         break
       else:
         r=wrd
     return(r)
  
def clss(arr):                                     #classified the postive negative sentiments
  p=0
  n=0
  z=0
  n1=len(arr)
  for i in arr:
    if i>0:
      p=p+1
    elif i<0:
      n=n+1
    else:
      z=z+1
    dfr=DataFrame([(n/n1)*100,(p/n1)*100,(z/n1)*100],index=["Negative","Positive","Nutral"],columns=["percent"])
  return(dfr)

def hot_words(df_n):        
  cr11b=df_n["Message"].values
  cr2b=[]
  for j in np.arange(len(cr11b)):
     txt=cr11b[j].lower()
     txt1=""
     for  i in txt.split(" "):
        if i not in (stop_words):
           txt1=txt1+"".join(i)+" "
     cr2b=np.append(cr2b,txt1)
  dfcr2b=DataFrame(cr2b,columns=["Message"]) 
  df_n=dfcr2b                     #gives the hot words for the given df for particular person
  from sklearn.decomposition import NMF
  nmf_model = NMF(n_components=7,random_state=23)
  tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
  dtm = tfidf.fit_transform(df_n['Message'])
  nmf_model.fit(dtm)
  for index,topic in enumerate(nmf_model.components_):
    r=([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
  return(r)
  

#time and proportion
def tm(df_tm,name):
  n=len(df_tm["Name"])
  df_tm.index=np.arange(n)
  d=[]
  for i in np.arange(1,len(df_tm["Time"])):
      if (df_tm["Name"][i]==name) and (df_tm["Name"][i-1]!=name):
          t1=df_tm["Time"][i]
          t0=df_tm["Time"][i-1]
          d1=t1-t0
          d=np.append(d,d1)
      else:
        continue
      ds=np.mean(d).seconds
      dd=(np.mean(d).days)*24*60
      dm=ds/60
  n1=0
  for i in np.arange(n):
    if (df_tm["Name"][i]==name):
        n1=n1+1
  p=(n1/n)*100
  return([dm+dd,p])

In [ ]:
#data_cleaning_and_separation

In [ ]:
train1=[]
for  i in np.arange(len(train)):
  if train[i][2:3]=="/":
    train1=np.append(train1,train[i])
train=train1

In [ ]:
t=[]
p2=[]
for i in np.arange(2,len(train)):
     x12,x11= train[i].split(" - ",1)
     y=int((x12)[6:8])
     m=int((x12)[3:5])           
     d=int(x12[0:2])            
     h=int(x12[9:11])              
     mt=int(x12[-5:-3])
     pm=x12[-2:]
     if pm=="pm":
        h=h+12
     d1=datetime.datetime(y, m, d, h, mt, 0)
     t=np.append(t,d1)
     p2=np.append(p2,x11)
name=[]
mes=[]
dt=[]
for i in np.arange(len(p2)):
   x1=p2[i].split(":",1)
   if len(x1)==2:
     name=np.append(name,x1[0])
     mes=np.append(mes,x1[1])
   else:
     dt=np.append(dt,i)
dft=DataFrame([t,p2]).T
dft=dft.drop(dt,axis=0)
dft["name"]=name
dft["message"]=mes
dft=dft.drop(1,axis=1)

In [ ]:
dft.columns=["Time","Name","Messages"]
dft.head(10)

,Time,Name,Messages
0,0020-06-24 20:54:00,Sudhanshu Ranjan,भई लोग ham roz gupta ar kapoor ke problems ba...
1,0020-06-24 20:55:00,Sudhanshu Ranjan,Arun bhai ar bhart bahi hamare doubts clear द...
2,0020-06-24 20:55:00,Shivam Gupta,Or koi bi query ho usko solve kiya jayega\n
3,0020-06-24 21:12:00,Bharat New,Okay .... No doubt\n
4,0020-06-24 21:12:00,Bharat New,Ham matlab kon kon ?\n
5,0020-06-24 21:15:00,Shivam Gupta,Jo jo group me h\n
6,0020-06-24 21:16:00,Akm,Well done\n
7,0020-06-24 21:16:00,Akm,Sb mil baat के karenge\n
8,0020-06-24 21:18:00,Bharat New,Ok\n
9,0020-06-26 18:36:00,Sudhanshu Ranjan,<Media omitted>\n


In [ ]:
names=dft["Name"].unique()
n2=[]
for i in names:
  n1=i.split()
  for j in n1:
      n2=np.append(n2,j.lower())
names=n2

In [ ]:
#some statistics analysis

In [ ]:
(mes_sep(dft,"Shivam Gupta")).head()

,Time,Message
0,0020-06-24 20:55:00,Or koi bi query ho usko solve kiya jayega\n
1,0020-06-24 21:15:00,Jo jo group me h\n
2,0020-06-26 20:09:00,Kaun sa third\n
3,0020-06-26 20:16:00,Ok\n
4,0020-07-01 13:59:00,Wait bahi usper pahuchate h to btate h\n


In [ ]:
#corpus_cleaning

In [ ]:
dfhe1=separator(he1)
dfhe2=separator(he2)
frames = [dfhe1, dfhe2]
dfhe=pd.concat(frames)
(dfhe.index)=np.arange(0,len(dfhe["english"]))

In [ ]:
#time_and_proportion

In [ ]:
tm(dft,"Akm")#+tm(dft,"Akm")[1]+tm(dft,"Shivam Gupta")[1]+tm(dft,"Sudhanshu Ranjan")[1]

[104.36666666666666, 23.981900452488688]

In [ ]:
#cr=(dft["Messages"].values)
df_cr=(mes_sep(dft,"Shivam Gupta"))
cr=df_cr["Message"].values
cr[0:5]

array([' Or koi bi query ho usko solve kiya jayega\n',
       ' Jo jo group me h\n', ' Kaun sa third\n', ' Ok\n',
       ' Wait bahi usper pahuchate h to btate h\n'], dtype=object)

In [ ]:
with open('s_h_w.txt','r') as txt:
    shw=txt.readlines()
shw1=separator(shw)

In [ ]:
cr1=[]
for i in np.arange(len(cr)):
  sen1=""
  sen=cr[i].split()
  n=len(sen)
  for j in np.arange(n):
     w=sen[j]
     lan=detect_language(w)
     if lan=="hin":
       nw=(transliterate(w, sanscript.DEVANAGARI, sanscript.HK))
       sen1=sen1+"".join(nw)+" "
     else:
       sen1=sen1+"".join(w)+" "
  cr1=np.append(cr1,sen1)
he_s1=[]
for  i in np.arange(len(he_s)):
    he_s1=np.append(he_s1,(he_s[i][:-1]))
he_s=he_s1
cr11=[]
for i in np.arange(len(cr1)):
   string = cr1[i]
   cleaned = " ".join(re.findall("[a-zA-Z]+", string))
   cr11=np.append(cr11,cleaned)
stop_words.update(he_s)
stop_words.update(names)
stop_words.update(['media omitted','media', 'omitted','https.','okh','feel'])    
cr2=[]
for j in np.arange(len(cr11)):
  txt=cr11[j].lower()
  txt1=""
  for  i in txt.split(" "):
     if i not in (stop_words):
      txt1=txt1+"".join(i)+" "
  cr2=np.append(cr2,txt1)
dfcr2=DataFrame(cr2)
d=[]
for i in np.arange(len(dfcr2[0])):
   if (dfcr2[0][i]=="") or (dfcr2[0][i]==" ") :
     d=np.append(d,i)
dfcr2=dfcr2.drop(d,axis=0)
cr22=dfcr2[0].values  

In [ ]:
#sentiment_part

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
cr3=[]
for j in np.arange(len(cr22)):
  txt=cr22[j].lower()
  txt1=""
  for  i in txt.split(): 
      if len(i)!=0:
         txt2=finder(shw1,i)
         txt1=txt1+"".join(txt2)+" "
  cr3=np.append(cr3,txt1)
sn=[]
for i in np.arange(len(cr3)):
  a = cr3[i]
  sc=sid.polarity_scores(a)
  sc1=(sc).get("compound")
  sn=np.append(sn,sc1)
dff=DataFrame([sn,cr3],index=["Score","Message"]).T

In [ ]:
df_bk=dff
#df_sr=dff
#df_sg=dff
#df_Akm=dff

In [ ]:
clss(df_bk["Score"])

,percent
Negative,5.555556
Positive,19.444444
Nutral,75.000000


In [ ]:
#topic_modelling

In [ ]:
hot_words(df_bk)

['wait', 'limit']

In [ ]:
#5_feelings

In [ ]:
with open('train.txt','r') as txt:
    train=txt.readlines()
x=[]
y=[]
for i in np.arange(len(train)-1):
  txt = train[i]
  x1,y1 = txt.split(";")
  x=np.append(x,x1)
  y=np.append(y,y1[:-1])
import random
from sklearn.utils import resample
import nltk
nltk.download('punkt')
df_im=DataFrame([y,x],index=["class","TF"]).T
df_e1=df_im[df_im["class"]=="love"]
df_e2=df_im[df_im["class"]=="fear"]
df_e3=df_im[df_im["class"]=="anger"]
df_e4=df_im[df_im["class"]=="joy"] 
df_e5=df_im[df_im["class"]=="sadness"]   
df_e11 = DataFrame([np.repeat("love",1000),random.sample(list(df_e1["TF"].values),1000)],index=["class","TF"]).T
df_e21 = DataFrame([np.repeat("fear",1000),random.sample(list(df_e2["TF"].values),1000)],index=["class","TF"]).T
df_e31 = DataFrame([np.repeat("anger",1000),random.sample(list(df_e3["TF"].values),1000)],index=["class","TF"]).T
df_e41 = DataFrame([np.repeat("joy",1000),random.sample(list(df_e4["TF"].values),1000)],index=["class","TF"]).T
df_e51 = DataFrame([np.repeat("sadness",1000),random.sample(list(df_e5["TF"].values),1000)],index=["class","TF"]).T
df_e=pd.concat([df_e11, df_e21,df_e31,df_e41,df_e51])
df_e.index=np.arange(len(df_e["class"].values))
nltk.download('wordnet')
x1=df_e["TF"].values
y1=df_e["class"].values
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x1, y1, test_size=.01, random_state=42)
x1=x_train
lemmatizer = WordNetLemmatizer()
xn=[]
for sen in x1:
   word_list = nltk.word_tokenize(sen)
   lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
   xn=np.append(xn,lemmatized_output)
 
 
cr=[]
for txt in xn:
   txt1=""
   for i in txt.split():
     if i not in (stop_words):
      txt1=txt1+"".join(i)+" "
   cr=np.append(cr,txt1)
 
y=[]
for i in y_train:
  if i=="anger":
    y=np.append(y,0)
  elif i=="fear":
    y=np.append(y,1)
  elif i=="joy":
    y=np.append(y,2)
  elif i=="love":
    y=np.append(y,3)
  else:
    y=np.append(y,4)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def model_tr():
    from sklearn.tree import DecisionTreeClassifier
    model=DecisionTreeClassifier()
    return(model)
pmodel = Pipeline([('tfidf', TfidfVectorizer(stop_words=stop_words)),
                       ('mo', model_tr())])
pmodel.fit(cr,y)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['https', 'mo', 'mon'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words={'', 'a', 'aa', 'aad', 'aadi',
                                             'aaj', 'aap', 'aapn', 'aapne',
                                             'aat'...
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('mo',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', ma

In [ ]:
#functional approach

In [ ]:
def data_cln(mat):
  train=mat
  train1=[]
  for  i in np.arange(len(train)):
    if train[i][2:3]=="/":
       train1=np.append(train1,train[i])
  train=train1
  t=[]
  p2=[]
  for i in np.arange(2,len(train)):
     x12,x11= train[i].split(" - ",1)
     y=int((x12)[6:8])
     m=int((x12)[3:5])           
     d=int(x12[0:2])            
     h=int(x12[9:11])              
     mt=int(x12[-5:-3])
     pm=x12[-2:]
     if pm=="pm":
        h=h+12
     d1=datetime.datetime(y, m, d, h, mt, 0)
     t=np.append(t,d1)
     p2=np.append(p2,x11)
  name=[]
  mes=[]
  dt=[]
  for i in np.arange(len(p2)):
    x1=p2[i].split(":",1)
    if len(x1)==2:
      name=np.append(name,x1[0])
      mes=np.append(mes,x1[1])
    else:
      dt=np.append(dt,i)
  dft=DataFrame([t,p2]).T
  dft=dft.drop(dt,axis=0)
  dft["name"]=name
  dft["message"]=mes
  dft=dft.drop(1,axis=1)
  dft.columns=["Time","Name","Messages"]
  return(dft)


def cln_sn(cr,he_s,names):
    cr1=[]
    for i in np.arange(len(cr)):
       sen1=""
       sen=cr[i].split()
       n=len(sen)
       for j in np.arange(n):
         w=sen[j]
         lan=detect_language(w)
         if lan=="hin":
            nw=(transliterate(w, sanscript.DEVANAGARI, sanscript.HK))
            sen1=sen1+"".join(nw)+" "
         else:
            sen1=sen1+"".join(w)+" "
       cr1=np.append(cr1,sen1)
    he_s1=[]
    for  i in np.arange(len(he_s)):
        he_s1=np.append(he_s1,(he_s[i][:-1]))
    he_s=he_s1

    cr11=[]
    for i in np.arange(len(cr1)):
         string = cr1[i]
         cleaned = " ".join(re.findall("[a-zA-Z]+", string))
         cr11=np.append(cr11,cleaned)
    stop_words.update(he_s)
    stop_words.update(names)
    stop_words.update(['media omitted','media', 'omitted','https.','okh']) 
    
    cr2=[]
    for j in np.arange(len(cr11)):
        txt=cr11[j].lower()
        txt1=""
        for  i in txt.split(" "):
           if i not in (stop_words):
             txt1=txt1+"".join(i)+" "
             cr2=np.append(cr2,txt1)
    dfcr2=DataFrame(cr2)

    d=[]
    for i in np.arange(len(dfcr2[0])):
       if (dfcr2[0][i]=="") or (dfcr2[0][i]==" ") :
          d=np.append(d,i)
    dfcr2=dfcr2.drop(d,axis=0)
    cr22=dfcr2[0].values 
    
    cr3=[]
    for j in np.arange(len(cr22)):
       txt=cr22[j].lower()
       txt1=""
       for  i in txt.split(): 
         if len(i)!=0:
            txt2=finder(shw1,i)
            txt1=txt1+"".join(txt2)+" "
       cr3=np.append(cr3,txt1)
    sn=[]
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    sid = SentimentIntensityAnalyzer()
    for i in np.arange(len(cr3)):
      a = cr3[i]
      sc=sid.polarity_scores(a)
      sc1=(sc).get("compound")
      sn=np.append(sn,sc1)
    dff=DataFrame([sn,cr3],index=["Score","Message"]).T
    return(dff)

In [ ]:
def fact(mat,name,he_s):
  dft=data_cln(mat)
  #names=dft["Name"].unique()
  names=dft["Name"].unique()
  n2=[]
  for i in names:
     n1=i.split()
     for j in n1:
        n2=np.append(n2,j.lower())
  names=n2
  cr=(mes_sep(dft,name))
  cr=cr["Message"].values
  df_bk=cln_sn(cr,he_s,name)
  mn=np.mean(df_bk["Score"])
  sd=np.sqrt(np.var(df_bk["Score"]))
  df_bk1=[]
  df_cr=df_bk["Message"].values
  for j in np.arange(len(df_cr)):
      txt=df_cr[j].lower()
      txt1=""
      for  i in txt.split(" "):
          if i not in (stop_words):
            txt1=txt1+"".join(i)+" "
            df_bk1=np.append(df_bk1,txt1)
  df_bk22=DataFrame(df_bk1,columns=["Message"])

  p=""
  s=""
  if mn>0:
    p="postive"
  elif mn<0:
    p="negative"
  else:
    p="nutral"

  if mn>mn+.3*sd or mn<mn-.3*sd:
    s="highly"
  else:
    s="slightly"
  c=s+" "+p
  t,p=tm(dft,name)
  em5=pmodel.predict(df_bk22["Message"].values)
  e1=0;e2=0;e3=0;e4=0;e5=0;
  for i in em5:
     if i==0:
       e1=e1+1
     elif i==1:
        e2=e2+1
     elif i==3:
        e3=e3+1
     elif i==4:
        e4=e4+1
     else :
        e5=e5+1
  en=len(em5)
  dfem=DataFrame([e1/en,e2/en,e3/en,e4/en,e5/en],index=["Anger","Fear","Joy","Love","Sadness"],columns=["proportion"])
  #return([clss(df_bk["Score"]),"\n Most frequentlt used words by ",name,"is",hot_words(df_bk)])
  print("\nThe average time taken by ",name," to reply is ",t,"min.")
  print("\n The percentage of ",name,"'s chats in total chat is ",p)
  print("\nThe nature of ",name,"using the  whatsapp chat data is given as ","\n\n" ,clss(df_bk["Score"]))
  print("\n",name,"is ",c,"person. And the mean score is ",mn)
  print("\n The most frquently used words by ",name, "are ",hot_words(df_bk22))
  #print("\n The furthur classification of sentiments can be \n",dfem)

In [ ]:
with open('WhatsApp Chat with Aliva.txt','r') as txt:
    train=txt.readlines()

In [ ]:
with open('s_h_w.txt','r') as txt:
    shw=txt.readlines()
shw1=separator(shw)
fact(train,"Aliva",he_s)


The average time taken by  Aliva  to reply is  19.033333333333335 min.

 The percentage of  Aliva 's chats in total chat is  47.55793805501408

The nature of  Aliva using the  whatsapp chat data is given as  

             percent
Negative   1.287976
Positive  36.879363
Nutral    61.832661

 Aliva is  slightly postive person. And the mean score is  0.1563223773473632

 The most frquently used words by  Aliva are  ['gc', 'ldki', 'kat', 'milti', 'aim', 'wla', 'pas', 'college', 'glti', 'ayi', 'test', 'lg', 'leke', 'cl', 'gyi']


In [ ]:
def data_cln(mat):
  train=mat
  train1=[]
  for  i in np.arange(len(train)):
    if train[i][2:3]=="/":
       train1=np.append(train1,train[i])
  train=train1
  t=[]
  p2=[]
  for i in np.arange(2,len(train)):
     x12,x11= train[i].split(" - ",1)
     y=int((x12)[6:8])
     m=int((x12)[3:5])           
     d=int(x12[0:2])            
     h=int(x12[9:11])              
     mt=int(x12[-5:-3])
     pm=x12[-2:]
     if pm=="pm":
        h=h+12
     d1=datetime.datetime(y, m, d, h, mt, 0)
     t=np.append(t,d1)
     p2=np.append(p2,x11)
  name=[]
  mes=[]
  dt=[]
  for i in np.arange(len(p2)):
    x1=p2[i].split(":",1)
    if len(x1)==2:
      name=np.append(name,x1[0])
      mes=np.append(mes,x1[1])
    else:
      dt=np.append(dt,i)
  dft=DataFrame([t,p2]).T
  dft=dft.drop(dt,axis=0)
  dft["name"]=name
  dft["message"]=mes
  dft=dft.drop(1,axis=1)
  dft.columns=["Time","Name","Messages"]
  return(dft)

In [ ]:
#train[8780].split(" - ",1)
j

8780

In [ ]:
#data_cln(train)
train1=[]
for  i in np.arange(len(train)):
  if train[i][2:3]=="/":
    train1=np.append(train1,train[i])
train=train1
t=[]
p2=[]
for i in np.arange(2,len(train)):
    x12,x11= train[i].split(" - ",1)
    j=i
print(j)

9591


In [ ]:
#thrash_box
p=[","," - ",": "]
x12,x11,x01= train[134].split(" - ")
#x12[4]     #name
#x12[1]     #time
#x12[4:]     #sentence
y=(x12)[6:8]
m=(x12)[3:5]           
d=x12[0:2]            
h=x12[9:11]              #ih hr in PM add 12 there
mt=x12[-5:-3]
pm=x12[-2:]
[y,m,d,h,mt,pm]




#pip install indic-transliteration
import indic_transliteration
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
data = 'हाल की एक स्टडी में पता चला है कि बीड़ी उद्योग में लगे लोगों की संख्या में जिसमें महिलाएं अधिक हैं, राष्ट्रीय स्तर पर इज़ाफा हुआ है लेकिन दक्षिणी सूबों में इसमें गिरावट दर्ज की गई है'
print(transliterate(data, sanscript.DEVANAGARI, sanscript.HK))



from sklearn.utils import resample
df_im=DataFrame([y,x],index=["class","TF"]).T
df_min1=df_im[df_im["class"]=="surprise"]
df_min2=df_im[df_im["class"]=="love"]
df_min3=df_im[df_im["class"]=="fear"]
df_min4=df_im[df_im["class"]=="anger"]
df_maj1=df_im[df_im["class"]=="joy"] 
df_maj2=df_im[df_im["class"]=="sadness"]   
df_min11 = resample(df_min1, 
                                 replace=True,     
                                 n_samples=2000,    
                                 random_state=250)
df_min22 = resample(df_min2, 
                                 replace=True,     
                                 n_samples=3000,    
                                 random_state=20)
df_min33 = resample(df_min3, 
                                 replace=True,     
                                 n_samples=5000,    
                                 random_state=212)
           
df_min44 = resample(df_min4, 
                                 replace=True,     
                                 n_samples=5000,    
                                 random_state=1120)
df_sm=pd.concat([df_min11, df_min22,df_min33,df_min44,df_maj1,df_maj2])
df_sm.index=np.arange(len(df_sm["TF"].values))




lemmatizer = WordNetLemmatizer()
xn=[]
for sen in x:
   word_list = nltk.word_tokenize(sen)
   lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
   xn=np.append(xn,lemmatized_output)


cr=[]
for txt in xn:
   txt1=""
   for i in txt.split():
     if i not in (stop_words):
      txt1=txt1+"".join(i)+" "
   cr=np.append(cr,txt1)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
xnn= vectorizer.fit_transform(cr) 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(cr)
vocabulary_size = len(tokenizer.word_counts)
from tensorflow.keras.preprocessing.text import one_hot
oh=[one_hot(words,vocabulary_size)for  words in cr]
sen_len=30
e_doc=pad_sequences(oh,padding="pre",maxlen=sen_len)






df_senti=DataFrame([clss(df_Akm["Score"])["percent"].values,clss(df_sg["Score"])["percent"].values,
           clss(df_bk["Score"])["percent"].values,clss(df_sr["Score"])["percent"].values],
          index=["Akm","shivam","bharat","SR"],columns=["negative","postive","nutral"]).T
mn=[np.mean(df_Akm["Score"]),np.mean(df_sg["Score"]),
                           np.mean(df_bk["Score"]),np.mean(df_sr["Score"])]
sd=[np.sqrt(np.var(df_Akm["Score"])),np.sqrt(np.var(df_sg["Score"])),
    np.sqrt(np.var(df_bk["Score"])),np.sqrt(np.var(df_sr["Score"]))]
df_st=DataFrame([mn,sd],columns=["Akm","shivam","bharat","SR"],index=["mean_score","sd_of_score"])
pd.concat([df_senti.T,df_st]).T